In [2]:
import pandas as pd

In [3]:
"""
I use a dataframe that I created earlier with the script "Game Scrape" to create the URLs needed to collect the data for 
plays.
"""
games = pd.read_csv("games.csv")

In [4]:
games.head()

,seasonStartYear,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Attend.,Notes,Start (ET),datetime,isRegular,game_id
0,1996,Sacramento Kings,85,Houston Rockets,96,16285.0,NaN,NaN,1996-11-01,1,1
1,1996,Los Angeles Clippers,97,Golden State Warriors,85,15593.0,NaN,NaN,1996-11-01,1,2
2,1996,Portland Trail Blazers,114,Vancouver Grizzlies,85,19193.0,NaN,NaN,1996-11-01,1,3
3,1996,Seattle SuperSonics,91,Utah Jazz,99,19911.0,NaN,NaN,1996-11-01,1,4
4,1996,New York Knicks,107,Toronto Raptors,99,28457.0,NaN,NaN,1996-11-01,1,5


In [10]:
"""
I add the abbreviation for the home team because I need it for creating the URLs.
"""

teamAbbreviation = {'Dallas Mavericks':'DAL', 'Phoenix Suns':'PHO', 'Boston Celtics':'BOS',
       'Portland Trail Blazers':'POR', 'New Jersey Nets':'NJN', 'Toronto Raptors':'TOR',
       'Los Angeles Lakers':'LAL', 'Utah Jazz':'UTA', 'Philadelphia 76ers':'PHI',
       'New York Knicks':'NYK', 'Minnesota Timberwolves':'MIN', 'Orlando Magic':'ORL',
       'San Antonio Spurs':'SAS', 'Sacramento Kings':'SAC', 'Atlanta Hawks':'ATL',
       'Seattle SuperSonics':'SEA', 'Washington Bullets':'WSB', 'Indiana Pacers':'IND',
       'Los Angeles Clippers':'LAC', 'Miami Heat':'MIA', 'Milwaukee Bucks':'MIL',
       'Charlotte Hornets':'CHO', 'Cleveland Cavaliers':'CLE', 'Houston Rockets':'HOU',
       'Denver Nuggets':'DEN', 'Vancouver Grizzlies':'VAN', 'Golden State Warriors':'GSW',
       'Chicago Bulls':'CHI', 'Detroit Pistons':'DET', 'Washington Wizards':'WAS',
       'Memphis Grizzlies':'MEM', 'New Orleans Hornets':'NOH', 'Charlotte Bobcats':'CHA',
       'New Orleans/Oklahoma City Hornets':'NOK', 'Oklahoma City Thunder':'OKC',
       'Brooklyn Nets':'BRK', 'New Orleans Pelicans':'NOP'}

games['abbreviation'] = games['Home/Neutral'].apply(lambda x: teamAbbreviation[x])

In [17]:
df = pd.DataFrame(columns=['time', 'event_away', 'point_away', 'score', 'point_home','event_home', 'game_id'])
for i, row in games.iterrows():
    year, month, day = row.datetime.split('-')
    url = "https://www.basketball-reference.com/boxscores/pbp/{}{}{}0{}.html".format(year, month, day, row.abbreviation)
    tables = pd.io.html.read_html(url)[0]['1st Q']
    tables.set_axis(['time', 'event_away', 'point_away', 'score', 'point_home','event_home'], axis=1, inplace=True)
    tables['game_id'] = row.game_id
    df = pd.concat([df, tables])

In [65]:
# df.to_csv("220224_96-14_plays_BR_collectedBy_PH.csv", index=False)

In [16]:
# df = pd.DataFrame(columns=['time', 'event_away', 'point_away', 'score', 'point_home','event_home', 'game_id'])
# for i, row in games[games.game_id>=22551].iterrows():
#     year, month, day = row.datetime.split('-')
#     url = "https://www.basketball-reference.com/boxscores/pbp/{}{}{}0{}.html".format(year, month, day, row.abbreviation)
#     tables = pd.io.html.read_html(url)[0]['1st Q']
#     tables.set_axis(['time', 'event_away', 'point_away', 'score', 'point_home','event_home'], axis=1, inplace=True)
#     tables['game_id'] = row.game_id
#     df = pd.concat([df, tables])


In [19]:
# df.to_csv("220224_14-21_plays_BR_collectedBy_PH.csv", index=False)

In [20]:
# old_df = pd.read_csv("220224_96-14_plays_BR_collectedBy_PH.csv")
# df = pd.concat([old_df, df])

In [23]:
# I save the uncleaned data in case I make mistakes later I can get back the data without rerunning the code.
df.to_csv('playByPlay_unclean.csv', index=False)

In [87]:
df = pd.read_csv("playByPlay_unclean.csv")

Things I need to do:
- Create play_id
- Change time to sec
- Create period variable
- Clean score variable

In [88]:
# Create play_id
df['play_id'] = [i+1 for i in range(len(df))]

In [89]:
def createTime(row):
    try:
        minute, sec = row.split(":")
        sec = sec.split('.')[0]
        return int(minute)*60 + int(sec)
    except:
        return row
    
df['periodTime'] = df.time.apply(createTime)
# I cut values that do not have a period time becuase these are headings for the tables on basketball-reference
df = df[df['periodTime'].apply(lambda x: isinstance(x, (int, float)))]
df.drop(columns=['time'], inplace=True)

In [92]:
"""
Create period variable:
- If game id changes --> new game starts
- If periodTime == 720 --> new period starts
"""

prevGame_id = 0
lst = []
for i, row in df.iterrows():
    if prevGame_id != row.game_id:
        lst.append(1)
        prevGame_id = row.game_id
        continue
    elif row.periodTime == 720:
        lst.append(lst[-1]+1)
        continue
    elif row.periodTime != 720:
        lst.append(lst[-1])
        continue
    else:
        lst.append("ERROR")
        
df['period'] = lst



In [93]:
def createScore(row):
    try:
        return [int(i) for i in row.score.split('-')]
    except:
        return None
    
pointsAway = []
pointsHome = []

for i, row in df.iterrows():
    point_lst = createScore(row)
    if point_lst != None:
        pointsAway.append(point_lst[0])
        pointsHome.append(point_lst[1])
    else:
        if row.period == 1 and row.periodTime == 720:
            pointsAway.append(0)
            pointsHome.append(0)
        else:
            pointsAway.append(pointsAway[-1])
            pointsHome.append(pointsHome[-1])
            
df['homePoints'] = pointsHome
df['awayPoints'] = pointsAway

df.drop(columns=['score'], inplace=True)

In [97]:
def cleanDuplicateValues(df, colName):
    return_lst = []
    for i, row in df.iterrows():
        if row.event_away == row.point_away:
            return_lst.append(None)
        elif row.point_away == None:
            return_lst.append(None)
        else:
            return_lst.append(row[colName])
    return return_lst

df['point_away'] = cleanDuplicateValues(df, 'point_away')
df['point_home'] = cleanDuplicateValues(df, 'point_home')
df['event_home'] = cleanDuplicateValues(df, 'event_home')


In [99]:
df.to_csv("play_data.csv", index=False)

In [105]:
df[['game_id', 'period']].drop_duplicates().groupby('period').count()

,game_id
period,
1,30250
2,30250
3,30250
4,30248
5,27495
6,1010
7,366
8,121
9,34
